# Object Detection Demo: Car Detection

This is a sample reference implementation to showcase object detection (car in this case) with single-shot detection (SSD) and Async API.
Async API improves the overall frame-rate of the application by not waiting for the inference to complete but continuing to do things on the host while inference accelerator is busy. 
Specifically, this code demonstrates two parallel inference requests by processing the current frame while the next input frame is being captured. This essentially hides the latency of frame capture.

## Overview of how it works
The inference executable (tutorial1) reads the command line arguments and loads a network and image from the video input to the Inference Engine (IE) plugin. 
A job is submitted to run the inference executable on a hardware accelerator (Intel® Core CPU, Intel® HD Graphics GPU, Intel® Core CPU, Intel® Movidius™ and/or Neural Compute Stick).
After the inference is completed, the output videos are appropriately stored in the /results directory, which can then be viewed within the Jupyter Notebook instance

## Demonstration objectives
* Video as input is supported using **OpenCV**
* Inference performed on edge hardware (rather than on the development node hosting this Jupyter notebook)
* **OpenCV** provides the bounding boxes, labels and other information
* Visualization of the resulting bounding boxes
* Demonstrate the Async API in action

## Step 0: Set Up

### 0.1: Import dependencies

Run the below cell to import Python dependencies needed for displaying the results in this notebook
(tip: select the cell and use **Ctrl+enter** to run the cell)

In [1]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *

### 0.2  (Optional-step): Original video without inference

If you are curious to see the input video, run the following cell to view the orignal video stream used for inference and object detection.

In [2]:
!ln -sf cars_1900.mp4 
videoHTML('Cars video', ['cars_1900.mp4'])

ln: 'cars_1900.mp4' and './cars_1900.mp4' are the same file


## Step 1: Using OpenVINO

First, let's try running inference on a single image to see how OpenVINO works.
We will be using OpenVINO's Inference Engine (IE) to locate vehicles on the road.
There are five steps involved in this task:

1. Create a Intermediate Representation (IR) Model using the Intel Model Optimizer
2. Choose a device and create IEPlugin for the device
3. Read the IRModel using IENetwork
4. Load the IENetwork into the Plugin
5. Run inference.

### 1.1 Creating IR Model

Intel Model Optimizer creates Intermediate Representation (IR) models that are optimized for different end-point target devices.
These models can be created from existsing DNN models from popular frameworks (e.g. Caffe, TF) using the Intel Model Optimizer. 

The Intel Distribution of OpenVINO includes a utility script `model_downloader.py` that you can use to download some common modes. Run the following cell to see the models available through `model_downloader.py`

In [3]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --print_all

Traceback (most recent call last):
  File "/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py", line 25, in <module>
    import requests
ModuleNotFoundError: No module named 'requests'


**Note** the '!' is a special Jupyter Notebook command that allows you to run shell commands as if you are in commannd line. So the above command will work straight out of the box on in a terminal (with '!' removed).

In this demo, we will be using the **mobilenet-ssd** model. This model can be downloaded with the following command.

In [4]:
#!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name mobilenet-ssd -o raw_models
#User might fail to download the models due to network issue
#Models already download to raw_models, user does not need to download.

The input arguments are as follows:
* --name : name of the model you want to download. It should be one of the models listed in the previous cell
* -o : output directory. If this directory does not exist, it will be created for you.

There are more arguments to this script and you can get the full list using the `-h` option.


With the `-o` option set as above, this command downloads the model in the directory `raw_models`, with the `.caffemodel` located at `raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel`

Now, let's convert this to the optimized model using the model optimizer.

In [5]:
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel \
--data_type FP32 \
-o models/mobilenet-ssd/FP32 \
--scale 256 \
--mean_values [127,127,127] 

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/kiwi/Document/aidevkit/val/AI_reference_samples/cpp/object_detection/raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel
	- Path for generated IR: 	/home/kiwi/Document/aidevkit/val/AI_reference_samples/cpp/object_detection/models/mobilenet-ssd/FP32
	- IR output name: 	mobilenet-ssd
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	[127,127,127]
	- Scale values: 	Not specified
	- Scale factor: 	256.0
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
Caffe specific parameters:
	- Path to Python Caffe* parser generated from caffe.proto: 	/opt/in

**Note** the above line is a single command line input, which spans 4 lines thanks to the backslash '\\', which is a line continuation character in Bash.

Here, the arguments are:
* --input-model : the original model
* --data_type : Data type to use. One of {FP32, FP16, half, float}
* -o : outout dirctory

This script also supports `-h` that will you can get the full list of arguments.

With the `-o` option set as above, this command will write the output to the directory `models/mobilenet-ssd/FP32`

There are two files produced:
```
models/mobilenet-ssd/FP32/mobilenet-ssd.xml
models/mobilenet-ssd/FP32/mobilenet-ssd.bin
```
These will be used later in the exercise.

We will also be needing the FP16 version of the model for the calculations on the MYRIAD architecture. Run the following cell to create it.

In [6]:
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel \
--data_type FP16 \
-o models/mobilenet-ssd/FP16 \
--scale 256 \
--mean_values [127,127,127] 

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/kiwi/Document/aidevkit/val/AI_reference_samples/cpp/object_detection/raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel
	- Path for generated IR: 	/home/kiwi/Document/aidevkit/val/AI_reference_samples/cpp/object_detection/models/mobilenet-ssd/FP16
	- IR output name: 	mobilenet-ssd
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	[127,127,127]
	- Scale values: 	Not specified
	- Scale factor: 	256.0
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
Caffe specific parameters:
	- Path to Python Caffe* parser generated from caffe.proto: 	/opt/in

### Step 1.2: Compile the code

The code in this demo is separated into two parts.
First part is responsible for reading the input stream and running the object detection inference workload on the stream. 
This part outputs Region Of Interest (ROI), in terms of coordinates, for each frame.
The source code for this part can be found in [main.cpp](./main.cpp), and the executable will be named "tutorial1".
Output ROI will be written into a text file, "ROIs.txt".

The second part reads the ROIs.txt file, and overlays boxes on each frame of the stream based on the coordinates.
Then the output video is written into a file. 
The source code for this step is in [ROI_writer.cpp](./ROI_writer.cpp).

We have provided a Makefile for compiling the examples. Run the following cell to compile the application.
(tip: use **crtl+enter** to run the cell)

In [7]:
!make

make: Nothing to be done for 'all'.


### Commandline flags

The two executables, tutorial1 and ROIwriter, take a number of commandline arguments.

Run the following cells to see the list of the available arguments: 

In [ ]:
!./tutorial1 -h

In [8]:
!./ROI_writer -h


[usage]
	ROIviewer [option]
	options:

		-h              Print a usage message
		-i <path>       Required. Path to input video file
		-ROIfile <path> Path to ROI file.
		-b #            Batch # to display.
		-l <path>       class labels file.
		-o <filename>       Output file path.
		-r <res>       (double) factor to cut reolution by; 2. will cut the resolution by half.
		-k <keepframe>       Writer will keep every <skipframe> frames.


## Step 2: Running the inference

Now we are ready to run the inference workload. We will run the workload on several edge compute nodes represented in the IoT DevCloud. We will send work to the edge compute nodes by submitting the corresponding non-interactive jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

**Note**: Currently, you are running this Notebook on a development node. On this system, you are alloated just one core on a large Xeon CPU. The purpose of this node is to develop code and run minimal jupyter notebooks, but it is not meant for compute jobs like deep learning inference. So we need to request additional resources from the cluster to run the inference, and this is done through the job queue.


The job file is written in Bash, and will be executed directly on the edge compute node.
For this example, we have written the job file for you in the notebook.
Run the following cell to write this in to the file "object_detection_job.sh" In this step, we will be submitting the workload as a job to the job queue. 

In [9]:

#!./tutorial1 -i cars_1900.mp4 \
#            -m models/mobilenet-ssd/FP32/mobilenet-ssd.xml \
#            -d CPU \
#            -o results\
#            -fr 3000 

# Converting the text output to a video
!./ROI_writer -i cars_1900.mp4 \
             -o results \
             -ROIfile results/ROIs.txt \
             -l pascal_voc_classes.txt \
             -r 2.0 # output in half res

opening cars_1900.mp4 batchnum 0
nuseclasses=20
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 28 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'


***Wait!***

Please wait for the inference jobs and video rendering complete before proceeding to the next step.

## Step 3: View Results

Once the jobs are completed, the queue system outputs the stdout and stderr streams of each job into files with names of the form

`obj_det_{type}.o{JobID}`

`obj_det_{type}.e{JobID}`

(here, obj_det_{type} corresponds to the `-N` option of qsub).

However, for this case, we may be more interested in the output video files. They are stored in mp4 format inside the `results/` directory.
We wrote a short utility script that will display these videos with in the notebook.
Run the cells below to display them.
See `demoutils.py` if you are interested in understanding further how the results are displayed in notebook.

In [10]:
videoHTML('Intel Core CPU',
          ['results/output.mp4'],
          'results/stats.txt')